# Data Split - Experimento

Este componente separa os dados em "treino", "validação" e "teste" podendo ter seus parâmetro customizados para separações mais eficientes. <br>
Essas separações podem ser feitas para deixar os conjuntos balanceados, portanto alterar o parâmetro "strat", ou garantir que mesmas observações(ID's iguais) não estejam em conjuntos diferentes, portantro alterar o parâmetro "split_ids".


## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [ ]:
# parameters
dataset = "/tmp/data/Iris.csv" #@param {type:"string"}
split_train = 80 #@param {type:"integer", label:"Porcentagem dedicada ao treino", description: "Tamanho em porcentagem do conjunto de treino, o restante será o conjunto de validação. Nesta porção não estará contida a porção dedicada ao teste."}
split_test = 30 #@param {type:"integer", label:"Porcentagem dedicada ao teste", description: "Tamanho em porcentagem do conjunto de teste, será subtraido do conjunto de validação."}

strat = None #@param {type:"feature",multiple:false,label:"Garante a mesma proporção de cada categoria da feature selecionada para os conjuntos de treino, validação e teste.", description: "Será usado a codificação ordinal se o atributo for categórico."}
split_ids = None #@param {type:"feature",multiple:false,label:"Atributo que restringe o mesmo ID de apareçer em mais de 1 conjunto, seja treino, validação ou teste.", description: "Não permitirá que IDs iguais fiquem em conjuntos diferentes."}
kfolds = None #@param {type:"integer", label:"Deseja usar validação cruzada será usada para validar algumas métricas do modelo? Se sim especifique a quantidade de dobras(folds).", description: " Recomenda-se o uso com poucas amostras."}
loo = None #@param {type:"integer", label:"Deseja usar a validação do estilo "leave-one-out"? Será usada para validar algumas métricas do modelo.", description: "Caso especial do validação crizada, recomenda-se o uso com pouquíssimas amostras."}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

df = pd.read_csv(dataset)
columns = df.columns

## Separando em conjunto de Treino, Validação e Teste

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit

if strat != None:
    balancy = df[strat]
else:
    balancy = None

if split_ids == None:
    
    # separo em treino/test
    X_train, X_test, idx1, idx2 = train_test_split(df, df.index, train_size=split_train/100, stratify = balancy)
    data = {'idx': np.concatenate([idx1,idx2]),
        'data-split': np.concatenate([ [0]*len(idx1),[1]*len(idx2)])
        }
    split = pd.DataFrame(data, columns = ['idx', 'data-split'])
    df["data-split"] = split.sort_values(by=['idx']).set_index('idx')["data-split"]
    
    #caso não tenha cv-> treino/val/teste
    if kfolds == None and loo == None:

        df_to_val_test = df.loc[df['data-split'] == 1].copy()
        df_train = df.loc[df['data-split'] == 0]
        
        if strat != None:
            balancy = X[strat]
    
        X_test, X_val, idx1, idx2 = train_test_split(df_to_val_test, df_to_val_test.index, test_size=split_test/100, stratify = balancy)
        data = {'idx': np.concatenate([idx1,idx2]),
                'data-split': np.concatenate([ [2]*len(idx1),[1]*len(idx2)])
                }
        split_aux = pd.DataFrame(data, columns = ['idx', 'data-split'])
        df_to_val_test["data-split"] = split_aux.sort_values(by=['idx']).set_index('idx')["data-split"]
        df = pd.concat([df_to_val_test, df_train])

else:
    
    #caso onde separo restringindo os ids -> treino/teste
    train_inds, test_inds = next(GroupShuffleSplit(train_size=split_train/100, n_splits=2, random_state = 42).split(df, groups=df[split_ids]))
    data = {'idx': np.concatenate([train_inds,test_inds]),
            'data-split': np.concatenate([ [0]*len(train_inds),[1]*len(test_inds)])
            }
    split = pd.DataFrame(data, columns = ['idx', 'data-split'])
    df["data-split"] = np.array(split.sort_values(by=['idx']).set_index('idx')["data-split"])

    if split_test != 0:

        #caso onde separo restringindo os ids -> treino/validação/teste  
        df_to_val_test = df.loc[df['data-split'] == 1].copy()
        df_train = df.loc[df['data-split'] == 0]
        test_inds, val_inds = next(GroupShuffleSplit(test_size=split_test/100, n_splits=2, random_state = 7).split(df_to_val_test, groups=df_to_val_test[split_ids]))
        data = {'idx': np.concatenate([test_inds,val_inds]),
                'data-split': np.concatenate([ [2]*len(test_inds),[1]*len(val_inds)])
                }
        split_aux = pd.DataFrame(data, columns = ['idx', 'data-split'])
        df_to_val_test["data-split"] = np.array(split_aux.sort_values(by=['idx']).set_index('idx')["data-split"])
        df = pd.concat([df_to_val_test, df_train])

In [ ]:
df.to_csv(dataset, index=False)
features_after_pipeline = df.columns
new_column_name = "data-split"

## Salva resultados da tarefa 

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.

In [ ]:
from joblib import dump

artifacts = {
    "columns": columns,
    "new_column_name" = new_column_name,
    "new_columns": df["data-split"],
    "features_after_pipeline": features_after_pipeline,
    "n_folds": kfolds,
    "use_loo": loo
}

dump(artifacts, "data-split.joblib")